In [ ]:
!pip install -q streamlit

In [ ]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-4iiw6_1i
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-4iiw6_1i
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 4.4 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=802825 sha256=bab3e958059b4ead6fde3d29ae54ba4ebe8f206e6fe945e1e9b4e477066303e0
  Stored in directory: /tmp/pip-ephem-wheel-cache-1l5c9bk9/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper


In [ ]:
!pip install --upgrade --no-deps --force-reinstall git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-hnurk03m
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-hnurk03m
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=802825 sha256=b115e6593406d1acd281d442dccbb1e6fcfc37070812806d807296396b2e68c1
  Stored in directory: /tmp/pip-ephem-wheel-cache-9jsb473n/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
  Attempting uninstall: openai-whisper
    Found existing installation: openai-whisper 20231117
    Uninstalling openai-whisper-20231117:
      Successfully uninstalled openai-whisper-20231117


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install soundfile sounddevice

In [ ]:
%%writefile app.py

import streamlit as st
import whisper
import pickle
import os
import librosa
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, Bidirectional, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import soundfile as sf

st.title("Voice authentication 🎵:")

audio_file = st.file_uploader("Upload Audio", type=["wav","mp3","flac"])
st.write(type(audio_file))

st.sidebar.header("Play the audio")
st.sidebar.audio(audio_file)

@st.cache_data
def load_model():
    try:
        with open("drive/MyDrive/KAGGLE/deepFake.pkl", "rb") as pickle_in:
            model = pickle.load(pickle_in)
        return model
    except FileNotFoundError:
        st.sidebar.error("Model file not found. Please make sure the file exists.")
        return None

# Function to extract features from an audio file
def extract_features(file_path, segment_length, file_name):
    try:
        # Load the audio file
        y, sr = librosa.load(file_path)
        print("sr: "+ str(sr))
        # Calculate the number of segments based on the segment length and audio length
        num_segments = int(np.ceil(len(y) / float(segment_length * sr)))
        print("num_segments: "+ str(num_segments))
        # Initialize a list to store the features for this file
        features = []

        # Extract features for each segment
        for i in range(num_segments):
            # Calculate start and end frame for the current segment
            start_frame = i * segment_length * sr
            end_frame = min(len(y), (i + 1) * segment_length * sr)
            # Extract audio for this segment
            y_segment = y[start_frame:end_frame]

            # Extract features
            chroma_stft = np.mean(librosa.feature.chroma_stft(y=y_segment, sr=sr))
            rms = np.mean(librosa.feature.rms(y=y_segment))
            spec_cent = np.mean(librosa.feature.spectral_centroid(y=y_segment, sr=sr))
            spec_bw = np.mean(librosa.feature.spectral_bandwidth(y=y_segment, sr=sr))
            rolloff = np.mean(librosa.feature.spectral_rolloff(y=y_segment, sr=sr))
            zcr = np.mean(librosa.feature.zero_crossing_rate(y_segment))
            mfccs = librosa.feature.mfcc(y=y_segment, sr=sr)
            mfccs_mean = np.mean(mfccs, axis=1)

            # Append the extracted features to the list
            features.append([chroma_stft, rms, spec_cent, spec_bw, rolloff, zcr, *mfccs_mean, file_name])

        return features
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

#Function to create dataset
def create_dataset_test(audio_file, segment_length):  # Modify to accept audio_file

    # Extract the file name without the extension
    file_name = os.path.splitext(audio_file.name)[0]  # Use audio_file.name for path

    # Extract features for the current file
    file_features = extract_features(audio_file.name, segment_length, file_name)
    if file_features:
        # Append features of all segments along with the label to the dataset
        for segment_features in file_features:
            feature_list.append(segment_features)

    # Create a DataFrame with the dataset
    df = pd.DataFrame(feature_list, columns=['chroma_stft', 'rms', 'spectral_centroid', 'spectral_bandwidth', 'rolloff', 'zero_crossing_rate', 'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10', 'mfcc11', 'mfcc12', 'mfcc13', 'mfcc14', 'mfcc15', 'mfcc16', 'mfcc17', 'mfcc18', 'mfcc19', 'mfcc20', 'origin_sample'])

    return df


def generate_csv(audio_file):

  # Length of the audio segments in seconds
  segment_length = 1  # for example, 1 second

  # Create the dataset
  dataset = create_dataset_test(audio_file.name, segment_length)

  # Save the dataset to a CSV file
  csv_output_path = 'audio.csv'
  dataset.to_csv(csv_output_path, index=False)

  print(f'Dataset created and saved to {csv_output_path}')
  return csv_output_path

def scale():
  # Load the dataset
  csv_output_path = 'drive/MyDrive/KAGGLE/audios.csv'
  dataset = pd.read_csv(csv_output_path)

  X = dataset.iloc[:, :-2].values  # Exclude 'origin_sample' and 'LABEL'
  y = dataset['LABEL'].values

  # Split the dataset into training and testing sets
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  # Scale the features
  from sklearn.preprocessing import MinMaxScaler
  scaler = MinMaxScaler()
  X_train_scaled = scaler.fit_transform(X_train.reshape(X_train.shape[0], -1))
  return scaler

# Function to record audio
def record_audio():
    st.write("Click the 'Record' button to start recording.")
    record_button = st.button("Record")

    if record_button:
        st.write("Recording... Click the 'Stop' button to stop.")
        audio_data = st.record(key="user_recording", format="wav", sample_rate=44100)
        st.write("Recording complete!")
        return audio_data

audio_file = record_audio()

if audio_file is not None:
    st.audio(audio_file, format="audio/wav", start_time=0)
    st.sidebar.success("Audio recorded!")

if st.sidebar.button("Load the model"):
  model = load_model()
  st.sidebar.success("Model loaded!")

if st.sidebar.button("Test Audio"):
  if audio_file is not None:
    st.sidebar.success("Processing audio...")
    csv_file = generate_csv(audio_file)

    # Use the model to make predictions on the dataset
    st.write("Making predictions with the model...")

    # Load the dataset
    dataset = pd.read_csv(csv_file)

    # Encode the labels
    label_encoder = LabelEncoder()
    dataset['LABEL'] = label_encoder.fit_transform(dataset['LABEL'])

    X_test = dataset.iloc[:, :-2].values  # Exclude 'origin_sample'

    scaler = scale()
    X_test_scaled = scaler.transform(X_test.reshape(X_test.shape[0], -1))
    X_test_scaled = X_test_scaled.reshape(X_test.shape)

    # Reshape the features to match the input shape expected by an RNN
    X_test_scaled = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

    y_pred = model.predict(X_test_scaled)
    y_pred_binary = np.round(y_pred)

    # Check if there are more 0s than 1s
    if np.sum(y_pred_binary == 0) > np.sum(y_pred_binary == 1):
      st.write("Prediction: fake")
    else:
      st.write("Prediction: real")


Overwriting app.py


In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 34.29.96.199


In [ ]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.029s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


In [ ]:
!streamlit run app.py &>/content/logs.txt &

In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 2.11s
your url is: https://rare-heads-shop.loca.lt
^C
